In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch

from bo_functions import ExactGPModel
from bo_functions import train_GP_model
from bo_functions import eval_GP_components
from bo_functions import calc_ei_point
from bo_functions import calc_ei_total_test

from bo_plotters import plot_hyperparams
from bo_plotters import ei_plotter_adv_test
from bo_plotters import ei_plotter
from bo_plotters import error_plotter_4D
from bo_plotters import y_plotter_4D
from bo_plotters import stdev_plotter_4D

In [2]:
#Pull x and Y data from CSV
#Pull x data from CSV
exp_data_doc = "exp_data.csv"
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1]
Yexp = exp_data[:,2]

n = len(Xexp)
dim = 3
# print(n)
Theta_True = np.array([1,-1])

In [3]:
#Create training and test data
train_data_doc = "train_3_in_data.csv"
train_data = np.array(pd.read_csv(train_data_doc, header=0,sep=","))
# print(train_data)
train_theta = train_data[:,1:dim]
train_p = torch.tensor(train_data[:,1:-1])
train_y = torch.tensor(train_data[:,-1])
# print(train_p)
# print(train_y)

In [4]:
# initialize likelihood and model
##Assumes a homoskedastic noise model p(y | f) = f + noise
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# We will use the simplest form of GP model, exact inference
#Defines our model in terms of the class parameters in bo_functions
model = ExactGPModel(train_p, train_y, likelihood)

In [5]:
#Set number of training iterations and train GP
iterations = 7000
train_GP = train_GP_model(model,likelihood, train_p, train_y, iterations, verbose=False)
noise_list = train_GP[0]
lengthscale_list = train_GP[1]
outputscale_list = train_GP[2]

KeyboardInterrupt: 

In [ ]:
#Plot hyperparameters vs iteration
noise_title = "Noise Hyperparameter"
lengthscale_title = "Lengthscale Hyperparameter"
outputscale_title = "Outputscale Hyperparameter"
plot_hyperparams(iterations, noise_list,noise_title)
plot_hyperparams(iterations, lengthscale_list,lengthscale_title)
plot_hyperparams(iterations, outputscale_list,outputscale_title)

In [ ]:
#Change GP Hyperparameters
# outputscale = torch.tensor([3.5078])
# lengthscale = torch.tensor([1.9804])
# noise = torch.tensor([10.0762])

# outputscale = torch.tensor([3.5])
# lengthscale = torch.tensor([2])
# noise = torch.tensor([10])

# model.likelihood.noise = noise
# model.covar_module.base_kernel.lengthscale =lengthscale
# model.covar_module.outputscale = outputscale

print("Noise Hyperparameter: ", model.likelihood.noise)
print("Lengthscale Hyperparameter: ", model.covar_module.base_kernel.lengthscale)
print("Outputscale Hyperparameter: ", model.covar_module.outputscale)

In [ ]:
# Get into evaluation (predictive posterior) mode
#Puts model in evaluation mode
model.eval()
#Puts likelihood in evaluation mode
likelihood.eval();

In [ ]:
#Create Meshgrid
p = 10
Theta1 = np.linspace(-2,2,p)
Theta2 = np.linspace(-2,2,p)

theta_mesh = np.array(np.meshgrid(Theta1,Theta2))
theta_space = torch.tensor(theta_mesh.T.reshape(-1,2))

In [ ]:
theta1_mesh = theta_mesh[0]
theta2_mesh = theta_mesh[1]

# for i in range(p):
#     for j in range(p):
#         print([theta1_mesh[i,j],theta2_mesh[i,j]])

In [ ]:
#Will compare the rigorous solution and approximation later (multidimensional integral over each experiment using a sparse grid)
#Calculate EI
EI_Components = eval_GP_components(p,n,Xexp,Yexp, theta_mesh, model, likelihood)
EI = EI_Components[0]
Error =EI_Components[1]
y_GP = EI_Components[2]
stdev_GP = EI_Components[3]
error_GP = EI_Components[4]

#Finds the index where sse is the smallest and finds which Theta combination corresponds to that value
argmin = np.array(np.where(np.isclose(Error, np.amin(Error),atol=1e-10)==True))
Theta_1_Opt = float(theta1_mesh[argmin[0],argmin[1]])
Theta_2_Opt = float(theta2_mesh[argmin[0],argmin[1]])
Theta_Opt_GP = np.array((Theta_1_Opt,Theta_2_Opt))
print("The GP predicts that Theta =",Theta_Opt_GP)

#calculates best theta value
argmax = np.array(np.where(np.isclose(EI, np.amax(EI),atol=1e-10)==True))
#     print(argmax)
Theta_1_Best = float(theta1_mesh[argmax[0],argmax[1]])
Theta_2_Best = float(theta2_mesh[argmax[0],argmax[1]])
Theta_Best = np.array((Theta_1_Best,Theta_2_Best))
print("The GP estimates the highest EI is at Theta = ",Theta_Best)

## Analysis of Expected Improvement
 - Expected Improvement is largest in corners
  - This is rational because you can't explore any further than the edges

In [ ]:
# print(Theta_True, Theta_GP_Opt)
ei_plotter(theta_mesh, EI, Theta_True, Theta_Opt_GP,train_p,plot_train=True)


#SHould I even plot y, stdev, and error? If so, would I plot them in 3D?

In [ ]:
EI_sing = calc_ei_total_test(p,n,Xexp,Yexp, theta_mesh, model, likelihood)[0]
Error =calc_ei_total_test(p,n,Xexp,Yexp, theta_mesh, model, likelihood)[1]
for i in range(n):    
    ei_plotter_adv_test(theta_mesh, EI_sing[i], Theta_True, train_p,Xexp[i],Theta_Opt_GP,plot_train=True)

In [ ]:
mesh_3D = np.meshgrid(Theta1,Theta2,Xexp)
stdev_plot = stdev_plotter_4D(mesh_3D,stdev_GP)

## Analysis of Standard Deviation
 - The GP estimates that the standard deviation is lowest at points that were directly tested
  - This can be rationalized by the way that the contour plot is drawn
 - Standard deviation is smallest away from the edges and larger towards them
  - This is rationalized by the fact that there are less neighbors that the GP is tested and trained with at the boundaries
 - The more points that get tested, the more the standard deviations will decrease

In [ ]:
error_plot = error_plotter_4D(mesh_3D,error_GP)

## Analysis of Error Magnitude
 - The GP emulator is most inaccurate when all values of $\bar{p}$ are at their maximum. 
  - In general, the GP is less accurate at extreme points, this is rationalized by the fact that there are less neighbors that the GP is tested and trained with at the boundaries
 - The GP emulator is most accurate when x is at it's maximum, but $\bar{\Theta}= 0$
  - This is rationalized by the fact that multiple terms become zero if any of the values of $\bar{p}$ are zero 
 - GP error is mostly very high, as more iterations are added, these will decrease

In [ ]:
y_plot = y_plotter_4D(mesh_3D,y_GP)

## Analysis of GP Emulator (Model y)
 - The GP emulator correctly captures that y increases as $\bar{p}$ increases. This tells us that this GP emulator model could be viable
  - The GP emulator correctly estimates where the lowest y is achieved, but not the actual value of y
  - The GP emulator slightly mistakes where the most positive value of y is, and does not predict the actual value of y
 - The model as it is is inaccurate, BO should increase the accuracy of the emulator